# BioLGCA - A Mesosocopic Modelling Framework for Collective Phenomena

## Introduction
This Juypter notebook serves as an introduction to the BioLGCA python package. It consists of lattice-gas cellular automaton models for biological systems in 1D and 2D, which can be adapted, simulated and analyzed. First, visualization and simulation methods are introduced. Next, already implemented LGCA models are presented and analyzed. We conclude with a demonstration on how to code and use your own interactions in the BioLGCA framework.

## Class initialization
### Calling the class
To envoke the package and obtain the appropriate class instance, the generic function `get_lgca` of the `base.py` file is used.

In [1]:
from lgca import get_lgca
lgca = get_lgca(geometry='hex')

Random walk interaction is used.


### Setting an interaction
Note that we did not specify the lattice dimensions or an interaction rule, so the default parameters are used. The default interaction is the random walk rule. We can use specific interactions and set the lattice dimensions using keyword arguments, like so

In [2]:
lgca = get_lgca(geometry='hex', dims=(10, 10), interaction='aggregation')

sensitivity set to beta =  2.0


We now used the aggregation rule, which needs a parameter called `beta`, which is set to 2 by default. It can also be set as a keyword argument. All available interactions can be printed using the class method `get_interactions`.

In [3]:
lgca.get_interactions()

['go_and_grow', 'go_or_grow', 'alignment', 'aggregation', 'random_walk', 'excitable_medium', 'nematic', 'persistant_motion', 'chemotaxis', 'contact_guidance']


The interaction can also be changed after initialization using the `set_interaction` method:

In [4]:
lgca.set_interaction(interaction='alignment', beta=3.0)

### Setting the initial state
By default, the initial state is a homogeneous state with constant mean density. The state of the lgca class instance is saved in the array `nodes`. For example, for a 1D LGCA of size 5, without rest channels and with a homogeneous initial state, where each channel is occupied with a probability $\rho = 0.1$, we use

In [5]:
lgca = get_lgca(restchannels=0, density=0.1, dims=5, geometry='lin', bc='refl')

AttributeError: 'LGCA_1D' object has no attribute 'nodes'

We can print the current state as

In [ ]:
print(lgca.nodes.astype(int))

## Simulating the system
A single time step without record can also be done with the method `timestep`:

In [ ]:
lgca.timestep()
lgca.nodes.astype(int)

To simulate the dynamics for longer times and record the steps we use the class method `timeevo`. It takes the keyword arguments `timesteps`, that sets the number of time steps to simualte. Also, all configurations can be recorded using `record=True`. To only record the density profile set `recorddens=True`. The total number of cells can be recorded with `recordN=True`. By default, only the density profile is recorded.

In [ ]:
lgca.timeevo(timesteps=100)
lgca.nodes.astype(int)

## Visualization
Visualization and plotting are different for 1D and 2D lattice geometries. The following section only applies to the 2D lattices, and the 1D equivalent is mentioned at the appropriate sections. 
### Plotting
#### Density profile
The python package offers several methods to visualize LGCA states. A simple method is plotting the current density profile $$n(r,k) = \sum_i s_i (r,k).$$ Consider a LGCA witha square lattice:

In [ ]:
lgca = get_lgca(geometry='square')
lgca.plot_density()

#### Flux
Sometimes it is more useful to plot the local flux $$ \mathbf{J}(r,k) = \sum_i s_i \mathbf{c_i}. $$ This can be achieved like follows:

In [ ]:
lgca.plot_flux()

Alternatively it is possible to plot a corresponding flow field:

In [ ]:
lgca.plot_flow()

#### Full configuration
For sketches, it is sometimes useful to plot the full lattice configuration. This is only advisable on small lattices. See the following example. Notice that the rest channels are depicted as circles, filled by a number, indicating the number of filled rest channels, while the occupied velocity channels are showns as black arrows.

In [ ]:
import numpy as np
nodes = np.zeros((2, 2, 6))
nodes[0, 0, :] = 1
nodes[1, 1, 4:] = 1
nodes[0, 1, :4] = 1
nodes[1, 0, 0] = 1
print(nodes)
lgca = get_lgca(geometry='square', density=0.25, nodes=nodes)
lgca.plot_config()

### Animation of dynamics
After recording configurations with the `timeevo` method, we can animate the dynamics (or plot it in a 2D plot, in the 1D case).

In [ ]:
lgca = get_lgca(geometry='hex', interaction='aggregation', dims=(10, 10), restchannels=0)
lgca.timeevo(record=True)

To animate the density we use the `animate_density` method like follows

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook
lgca.animate_density(interval=100)

Similarly, we can animate the flux, flow, and configuration:

In [ ]:
lgca.animate_flux(interval=100)

In [ ]:
lgca.animate_flow()

In [ ]:
lgca.animate_config()

### Live animations
Using the same logic as before, we can also animate the system on-the-fly. This is especially useful during interaction design and debugging.

In [ ]:
lgca.live_animate_density()

In [ ]:
lgca.set_interaction(interaction='alignment')
ani = lgca.live_animate_flux()

We can also change the boundary conditions, e.g. we change from the default periodic boundaries to reflecting boundaries here.

In [ ]:
lgca.set_bc('refl')
lgca.live_animate_config()

## Examples
### Alignment

In [ ]:
lgca = get_lgca(geometry='hex', interaction='alignment', bc='refl')
lgca.live_animate_flux()

### Aggregation

In [ ]:
lgca = get_lgca(interaction='aggregation', density=0.5)
lgca.live_animate_density()

### Nematic interaction

In [ ]:
lgca = get_lgca(interaction='nematic')
lgca.live_animate_flux()

### Chemotaxis

In [ ]:
lgca = get_lgca(interaction='chemotaxis')
lgca.live_animate_density()

### Persistent movement

In [ ]:
lgca = get_lgca(interaction='persistent_motion')
lgca.nodes[...] = 0
lgca.update_dynamic_fields()
lgca.nodes[10, 10, 1] = 1
lgca.live_animate_density()

### Nematic contact guidance

In [ ]:
lgca = get_lgca(interaction='contact_guidance')
lgca.nodes[...] = 0
lgca.update_dynamic_fields()
lgca.nodes[10, 10, 1] = 1
lgca.live_animate_density()

### Go and grow

In [ ]:
lgca = get_lgca(interaction='go_and_grow', restchannels=6)
lgca.nodes[...] = 0
lgca.update_dynamic_fields()
lgca.nodes[lgca.lx//2, lgca.ly//2, -1] = 1
lgca.live_animate_density()

### Go or grow

In [ ]:
lgca = get_lgca(interaction='go_or_grow', restchannels=6, kappa=4.)  # change parameter kappa from 4 to -4; what happens?
lgca.nodes[...] = 0
lgca.update_dynamic_fields()
lgca.nodes[lgca.lx//2, lgca.ly//2, :] = 1
lgca.live_animate_density()

### Excitable medium

In [ ]:
lgca = get_lgca(interaction='excitable_medium', restchannels=20, N=20, bc='refl')
lgca.nodes[...] = 0
lgca.nodes[:lgca.lx//2, :, :lgca.velocitychannels] = 1
lgca.nodes[:, :lgca.ly//2, lgca.velocitychannels:] = 1
lgca.live_animate_density(channels=slice(0, lgca.velocitychannels), vmax=lgca.velocitychannels)

## Using a custom interaction rule
Here, we will show how to use your own interaction rule, without changing the source code of the python package. The interaction rule should have the syntax `interaction(lgca)`, where `lgca` is an instance of any LGCA class, and it should change the attribute `lgca.nodes` to the post-interaction state. In this example, we will design a rule, that will make the particles align OR let the form aggregates.

In [ ]:
from bisect import bisect_left
from random import random


def rest_or_align(lgca):
    newnodes = np.zeros_like(lgca.nodes)
    resting = lgca.nodes[..., lgca.velocitychannels:].sum(-1)
    resting = lgca.nb_sum(resting)  # calc surrounding number of occupied rest channels for each node
    g = lgca.calc_flux(lgca.nodes)  
    g = lgca.nb_sum(g)  # calc cumulative flux around each node
    relevant = (lgca.cell_density[lgca.nonborder] > 0) & (lgca.cell_density[lgca.nonborder] < lgca.K)
    coords = [a[relevant] for a in lgca.nonborder]  # get relevant coordinates, to increase performance
    for coord in zip(*coords):
        n = lgca.cell_density[coord]
        permutations = lgca.permutations[n]  # pre-calculated unique permutations for cell number n
        j = lgca.j[n]  # and respective fluxes
        n_rest = permutations[:, lgca.velocitychannels:].sum(-1)  # and respective resting particles (could also be pre-calculated to speed up)
        weights = np.exp(lgca.beta * np.einsum('i,ij', g[coord], j) + lgca.alpha * n_rest * resting[coord]).cumsum()
        ind = bisect_left(weights, random() * weights[-1])
        newnodes[coord] = permutations[ind]
        
    lgca.nodes = newnodes

lgca = get_lgca(interaction='alignment', restchannels=1)
lgca.alpha = 2.0  # set the required alpha parameter as class attribute
lgca.interaction = rest_or_align  # set the interaction function

In [ ]:
lgca.live_animate_flux()

In [ ]:
from lgca import get_lgca
from random import random
from bisect import bisect_left
import numpy as np
%matplotlib notebook

In [ ]:
def imitate(lgca):
    newnodes = np.zeros_like(lgca.nodes)
    relevant = (lgca.cell_density[lgca.nonborder] > 0) & (lgca.cell_density[lgca.nonborder] < lgca.K)
    coords = [a[relevant] for a in lgca.nonborder]  # get relevant coordinates, to increase performance
    g = lgca.calc_flux(lgca.nodes)
    nbs = lgca.nb_sum(lgca.cell_density)
    g_nbs = lgca.nb_sum(g)
    v_nbs = np.divide(g_nbs, nbs[..., None], where=nbs[..., None]>0, out=np.zeros_like(g_nbs))
    for coord in zip(*coords):
        n = lgca.cell_density[coord]
        permutations = lgca.permutations[n]  # pre-calculated unique permutations for cell number n
        j = lgca.j[n]
        v = j / n
        v_nb = v_nbs[coord]
        n_rest = permutations[:, lgca.velocitychannels:].sum(-1)  # and respective resting particles (could also be pre-calculated to speed up)
        weights = np.exp(-lgca.beta * nbs[coord]* np.linalg.norm(v_nb[..., None] - v, axis=0) + lgca.alpha * n_rest).cumsum()
        ind = bisect_left(weights, random() * weights[-1])
        newnodes[coord] = permutations[ind]


    lgca.nodes = newnodes


In [ ]:
lgca = get_lgca(interaction='alignment', restchannels=1, beta=5, density=0.1)
lgca.alpha = -5  # set the required alpha parameter as class attribute
lgca.interaction = imitate  # set the interaction function

In [ ]:
lgca.live_animate_flux()